## Creating a chatbot with OpenAI and gpt-3.5-turbo

In [ ]:
import gradio as gr
import openai


from dotenv import load_dotenv, find_dotenv
import os

load_dotenv()

# this openai api key should be saved in a local file named ".env". It should also be put in the .gitignore to avoid sharing secrets
openai.api_key =  os.environ.get("OPENAI_API_KEY")

global init_state
init_state = [{"role": "system", "content": "You are a helpful assistant."}]

def chat_prompt(prompt, history, state):
    print(history, state)
    state = state if len(state) > 1 else init_state
    history = history or []
    dialog = {"role":"user", "content":prompt}
    state.append(dialog)
    return "", history + [[prompt, None]], state



def bot(history, state):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=state,
        temperature=1,
        n=1,
        stream=False,
        max_tokens=200
    )


    content = response['choices'][0]['message']['content']
    history += [[None,content]]
    state.append({"role": "assistant", "content":content})

    return history,state


block = gr.Blocks()

with block:
    gr.Markdown("""<h1><center> ChatGPT simple clone</center></h1>""")
    gr.Markdown("""this should work mostly like chat gpt, but might error out after a while due to history. If that happens try refreshing the page. Max tokens on a response is set to 200.""")
    chatbot = gr.Chatbot()
    message=gr.Textbox(placeholder="type a message to get started...")
    clear = gr.Button("Clear Conversation")
    state = gr.State(value = init_state)
    
    # add .then to prune the chat if it gets too long as a 3rd step?
    message.submit(chat_prompt, inputs=[message, chatbot, state], outputs=[message, chatbot, state], queue=False).then(bot, [chatbot,state], [chatbot,state])
    clear.click(lambda: None, None, chatbot, queue=False)#.then(gr.State(), init_state, state, queue=False)

block.launch()

